## Databrics Autoloader 

In [ ]:
import logging
from pyspark.sql.functions import col, size, split, input_file_name 
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, DateType, TimestampType 

logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)


Below I will create a function that will have the autloader logic to use.   This code will be reuse to load different files into the unity catalog using autoloader

In [ ]:
def autoloader(
        datasource, 
        format, 
        checkpoint,
        table, 
        schema, 
        file
):
    logger.info("start reading data using readstream")

    query = (
        spark.readstream
        .format("cloudFiles")
        .option("cloudFiles.format", format)
        .option("cloudFiles.schemaLocation", checkpoint) # this is where autoloader will house metadata 
        .option("cloudFiles.includeExistingFiles", "true") # Includes existing files in the file path 
        .option("cloudFiles.useIncrementalListing", "true") # use incremental listing to detect new files 
        .option("recursiveFileLookup", "true") # enable recursive file lookup 
        .option("header", "true")
        .opiton("pathGlobFilter", f"*{file}*") # this is a file patter that will be use to filter file name
        .schema(schema)
        .load(datasource)
    )

    # Running transofmation on streamed data